In [56]:
!pip install PyPDF2 sentence_transformers chromadb langchain langchain-groq

In [57]:
from langchain_groq import ChatGroq

llm = ChatGroq(
        model="llama-3.1-70b-versatile",
        api_key="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",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    )

In [58]:
import PyPDF2
import os
from docx import Document

PDF_FOLDER = 'Documents'

def extract_text_from_pdf_with_pages(pdf_path, pdf_file):
    global text_by_page
    try:
        with open(pdf_path, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            for page_number, page in enumerate(reader.pages):
                text_by_page.append((page_number + 1, page.extract_text(),pdf_file))  # Page numbers are 1-indexed
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
    return text_by_page

def extract_text_from_docx(docx_path, docx_file):
    global text_by_page
    try:
        doc = Document(docx_path)
        full_text = []
        for paragraph in doc.paragraphs:
            full_text.append(paragraph.text)
        text_by_page.append((1, "\n".join(full_text), docx_file))  # Treat as a single "page"
    except Exception as e:
        print(f"Error reading {docx_path}: {e}")
    return text_by_page


In [59]:
def file_processing(file_name):
    if file_name.endswith(".pdf"):  # Process only PDF files
        pdf_path = os.path.join(PDF_FOLDER, file_name)
        print(f"Processing {file_name}...")

        # Extract text
        #text = extract_text_from_pdf(pdf_path)
        text_by_page = extract_text_from_pdf_with_pages(pdf_path,file_name)
            
        if not text_by_page:  # Skip empty or unreadable files
            print(f"Skipping {pdf_file}: No text extracted.")
            return
    elif file_name.endswith(".docx"):  # Process DOCX files
        docx_path = os.path.join(PDF_FOLDER, file_name)
        print(f"Processing DOCX {file_name}...")

        # Extract text from DOCX
        text_by_page = extract_text_from_docx(docx_path, file_name)

        if not text_by_page:  # Skip empty or unreadable files
            print(f"Skipping {file_name}: No text extracted.")
            return

In [60]:
text_by_page = []
# Iterate over PDFs in the folder
for file_name in os.listdir(PDF_FOLDER):
    file_processing(file_name)
    

Processing DOCX dynamic_data.docx...
Processing Finetunung_LLM.pdf...
Processing genai-principles.pdf...


In [61]:
print(text_by_page)

[(1, "Resolving challenges in a Retrieval-Augmented Generation (RAG) pipeline at the enterprise level involves a combination of advanced technical implementations, infrastructure optimization, and organizational practices. Here’s how each challenge can be addressed:\n\n1. Retriever Efficiency\nChallenge: Retrieving relevant documents from large corpora can be computationally expensive and time-intensive.\n\nSolutions:\nEfficient Indexing:\nUse high-performance vector search engines like FAISS, Pinecone, or Weaviate to create optimized indexes.\nEmploy hierarchical indexing or partitioning to speed up retrieval.\nSparse + Dense Fusion:\nCombine traditional methods like BM25 with dense vector retrieval for a hybrid approach that balances efficiency and accuracy.\nPre-filtering:\nApply metadata-based filtering (e.g., categories, tags) to reduce the search space before dense similarity search.\nAsynchronous Updates:\nIncrementally update the knowledge base with new documents using asynchro

In [62]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
CHUNK_SIZE=10000
OVERLAP_SIZE=1000
def chunk_text_with_pages(text_by_page, chunk_size=CHUNK_SIZE, overlap_size=OVERLAP_SIZE):
    chunks_with_metadata = []
    splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n","\n"],  # List of separators based on requirement (defaults to ["\n\n", "\n", " "])
    chunk_size = chunk_size,  # size of each chunk created
    chunk_overlap  = overlap_size,  # size of  overlap between chunks in order to maintain the context
    length_function = len  # Function to calculate size, currently we are using "len" which denotes length of string however you can pass any token counter)
    )
    for page_number, text,pdf_file, in text_by_page:
        chunks = splitter.split_text(text)
        for chunk in chunks:
            chunks_with_metadata.append({"chunk": chunk, "page_number": page_number, "file_name": pdf_file})
    return chunks_with_metadata


In [63]:
# Chunk text using LangChain
chunks_with_metadata = chunk_text_with_pages(text_by_page, CHUNK_SIZE, OVERLAP_SIZE)

In [64]:
chunks_with_metadata

[{'chunk': "Resolving challenges in a Retrieval-Augmented Generation (RAG) pipeline at the enterprise level involves a combination of advanced technical implementations, infrastructure optimization, and organizational practices. Here’s how each challenge can be addressed:\n\n1. Retriever Efficiency\nChallenge: Retrieving relevant documents from large corpora can be computationally expensive and time-intensive.\n\nSolutions:\nEfficient Indexing:\nUse high-performance vector search engines like FAISS, Pinecone, or Weaviate to create optimized indexes.\nEmploy hierarchical indexing or partitioning to speed up retrieval.\nSparse + Dense Fusion:\nCombine traditional methods like BM25 with dense vector retrieval for a hybrid approach that balances efficiency and accuracy.\nPre-filtering:\nApply metadata-based filtering (e.g., categories, tags) to reduce the search space before dense similarity search.\nAsynchronous Updates:\nIncrementally update the knowledge base with new documents using as

In [65]:
from sentence_transformers import SentenceTransformer
MODEL_NAME= "all-mpnet-base-v2"
def generate_embeddings(chunks, model_name=MODEL_NAME):
    model = SentenceTransformer(model_name)
    embeddings = model.encode(chunks, show_progress_bar=True)
    return embeddings


In [66]:
embeddings=generate_embeddings(chunks_with_metadata)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [67]:
embeddings

array([[ 0.06741509,  0.03627042, -0.00124236, ...,  0.0187794 ,
        -0.04226869, -0.01900765],
       [ 0.08236661,  0.02293764, -0.02640497, ...,  0.03725887,
        -0.12297844, -0.01474249],
       [ 0.0853631 ,  0.01932502, -0.02593875, ...,  0.04847268,
        -0.08994223, -0.06346259],
       ...,
       [ 0.05958821,  0.02113394, -0.00898292, ...,  0.03281295,
        -0.06478857, -0.0457931 ],
       [ 0.06739228,  0.05418719, -0.02677693, ...,  0.03942776,
        -0.0771951 , -0.0341876 ],
       [ 0.01247031,  0.01009415, -0.04380452, ..., -0.02829326,
        -0.05914651, -0.0624537 ]], dtype=float32)

In [68]:
import chromadb
from chromadb.config import Settings

CHROMA_DB_DIR = "chroma_db"  # Directory to store the Chroma database
COLLECTION_NAME="RAG_embeddings"

client = chromadb.Client(Settings(persist_directory=CHROMA_DB_DIR))
if COLLECTION_NAME not in client.list_collections():
    collection = client.create_collection(name=COLLECTION_NAME)
else:
    collection = client.get_collection(name=COLLECTION_NAME)

# Prepare metadata and upload to ChromaDB
for i, (chunk_metadata, embedding) in enumerate(zip(chunks_with_metadata, embeddings)):
    metadata = {
        "filename": chunk_metadata["file_name"],
        "chunk_index": i,
        "page_number": chunk_metadata["page_number"],
        "text": chunk_metadata["chunk"]
    }
    pdf_file=chunk_metadata["file_name"]
    collection.add(
        
        ids=[f"{pdf_file}_chunk_{i}"],
        embeddings=[embedding],
        metadatas=[metadata]
    )

UniqueConstraintError: Collection RAG_embeddings already exists

In [ ]:
print(collection)

In [ ]:
COLLECTION_NAME = "RAG_embeddings"  # Replace with your collection name

if COLLECTION_NAME in [col.name for col in client.list_collections()]:
    collection = client.get_collection(name=COLLECTION_NAME)
    
    # Fetch and print details about the collection
    print(f"Collection: {COLLECTION_NAME}")
    
    # Fetch metadata for all items
    items = collection.get(include=["metadatas"])
    print("Metadata and Documents in the collection:")
    for metadata in items["metadatas"]:
        print(f"Metadata: {metadata}")
       
else:
    print(f"Collection {COLLECTION_NAME} does not exist.")


In [ ]:
def get_existing_files(directory):
    """Return a set of files in the directory."""
    return {file for file in os.listdir(directory) if os.path.isfile(os.path.join(directory, file))}

def detect_changes(directory, known_files):
    """Detect new and deleted files in the directory."""
    current_files = get_existing_files(directory)
    new_files = current_files - known_files
    deleted_files = known_files - current_files
    return new_files, deleted_files

In [ ]:
known_files = get_existing_files(PDF_FOLDER)

In [ ]:
known_files

In [ ]:
new_files, deleted_files = detect_changes(PDF_FOLDER, known_files)
print(new_files)
print(deleted_files)

In [ ]:
if new_files:
    text_by_page=[]
    print(f"New files detected: {new_files}")
    for file_name in new_files:
        file_processing(file_name)
        

In [ ]:
print(text_by_page)

In [ ]:
updated_chunks_with_metadata=chunk_text_with_pages(text_by_page, chunk_size=CHUNK_SIZE, overlap_size=OVERLAP_SIZE)

In [ ]:
updated_chunks_with_metadata

In [ ]:
new_embeddings=generate_embeddings(updated_chunks_with_metadata)
new_embeddings

In [ ]:
for i, (chunk_metadata, embedding) in enumerate(zip(updated_chunks_with_metadata, new_embeddings)):
    metadata = {
        "filename": chunk_metadata["file_name"],
        "chunk_index": i,
        "page_number": chunk_metadata["page_number"],
        "text": chunk_metadata["chunk"]
    }
    pdf_file=chunk_metadata["file_name"]
    collection.add(
        
        ids=[f"{pdf_file}_chunk_{i}"],
        embeddings=[embedding],
        metadatas=[metadata]
    )

In [ ]:
COLLECTION_NAME = "RAG_embeddings"  # Replace with your collection name

if COLLECTION_NAME in [col.name for col in client.list_collections()]:
    collection = client.get_collection(name=COLLECTION_NAME)
    
    # Fetch and print details about the collection
    print(f"Collection: {COLLECTION_NAME}")
    
    # Fetch metadata for all items
    items = collection.get(include=["metadatas"])
    print("Metadata and Documents in the collection:")
    for metadata in items["metadatas"]:
        print(f"Metadata: {metadata['filename']}")
       
else:
    print(f"Collection {COLLECTION_NAME} does not exist.")
